Pueden encontrar el link al repositorio [aquí](https://github.com/canel1125/NLP_Amazon_Review)

# Retomamos el modelo del proyecto anterior.

## Spacy

Utilizamos Spacy y sus funciones para lemmatizar, vectorizar y modelar.

Importamos de nuevo el dataset para poder trabajar en limpio y ver como se comporta si lo normalizamos y transformamos con Spacy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import spacy
import nltk
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

import string

In [ ]:
dataset = pd.read_json('dataset_amazon/dataset_es_train.json', lines = True)
dataset

In [ ]:
dataset.drop(['language','reviewer_id','product_id','review_id','product_category'],axis = 1,inplace=True)
dataset

In [ ]:
from spacy import displacy
nlp = spacy.load("es_core_news_sm") # Cargamos la versión en español

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#### Armamos la funciones para limpiar y procesar las oraciones

In [ ]:
# Libreria para reemplzar caracteres
import re
from nltk import SnowballStemmer

# Importamos la función que nos permite Stemmizar de nltk y definimos el stemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer('spanish')


# Traemos nuevamente las stopwords
stopwords = nltk.corpus.stopwords.words('spanish')
stopwords.remove('muy')
stopwords.remove('nada')
stopwords.remove('poco')
stopwords.remove('no')

In [ ]:
#Funcion que remueve acentos de palabras
def remover_acentos(palabra):
    #Reglas o letras a cambiar
    reglas = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", "n"),
    )
    for a, b in reglas:
        palabra = palabra.replace(a, b).replace(a.upper(), b.upper())
    return palabra

In [ ]:
#Funcion que tokeniza y normaliza las oraciones

def token_cleaner(oracion):
    doc = nlp(oracion)
    temp=[]
    for token in doc:
        # Vamos a reemplzar los caracteres que no sean letras por espacios
        token=remover_acentos(token.lemma_)
        token=re.sub("[^a-zA-Z]"," ",str(token))
        # Pasamos todo a minúsculas
        token=token.lower()
        # Tokenizamos para separar las palabras del titular
        token=nltk.word_tokenize(token)
        # Eliminamos las palabras de menos de 3 letras
        token = [palabra for palabra in token if len(palabra)>=2]
        # Sacamos las Stopwords
        token = [palabra for palabra in token if not palabra in stopwords ]

        ## Hasta acá Normalizamos, ahora a stemmizar
        
        # Por ultimo volvemos a unir el titular
        token="".join(token)

        # Agregamos a la lista la review
        temp.append(token)
        
    temp = list(filter(None, temp))
    
    return(temp)


In [ ]:
punct = string.punctuation

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [ ]:
#Funcion para poder graficar la matriz de confusión
from sklearn.metrics import confusion_matrix
import seaborn as sns

def confusion(ytest,y_pred):
    stars=["1","2"]
    cm=confusion_matrix(ytest,y_pred)
    f,ax=plt.subplots(figsize=(5,5))
    sns.heatmap(cm,annot=True,linewidth=.5,linecolor="r",fmt=".0f",ax=ax)
    plt.xlabel("y_pred")
    plt.ylabel("y_true")
    plt.show()

    return

#### Empezamos a definir el modelo

In [ ]:
#Como calsificador vamos usar un SVC y para tokenizar TFIDF, que debería tener un mejor desempeño
from sklearn.svm import LinearSVC
tfidf = TfidfVectorizer(tokenizer = token_cleaner)
classifier = LinearSVC()

Dividimos, entrenamos y predecimos como siempre

In [ ]:
X = dataset['review_title']
y = dataset['stars']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

Armamos una matriz de confusión para ver como se desempeña

In [ ]:
confusion(y_test,y_pred)

Imprimimos el valor para poder comparar con los siguientes modelos que vamos a entrenar cuando reduscamos la cantidad de variables a predecir

In [ ]:
clf.score(X_test,y_test)

Probamos con unos titulos basicos a ver como clasifica a mano

In [ ]:
titulosdeprueba = [["muy malo",0],["malo",0],["bueno",0], ["medio malo",0],["excelente",0], ["si me sirvio para lo que es",0], ["Buen precio calidad",0]]

for titulo in titulosdeprueba:
    titulo[1]=clf.predict([titulo[0]])[0]

In [ ]:
tabla_titulosdeprueba = pd.DataFrame(titulosdeprueba, columns = ["Review","Puntaje predicho"])

tabla_titulosdeprueba

# Binarizacion del problema y reducción de error

En esta etapa buscaremos reducir el error y mejorar el score del modelo del anterior proyecto binarizando el problema. Para esto, en lugar de predecir un puntaje, predecirá si la review recomienda el producto o no.

In [ ]:
#Separamos el dataset en 2
dataset.loc[dataset[dataset['stars']< 3].index,'recomendable']=0
dataset.loc[dataset[dataset['stars']>= 3].index,'recomendable']=1

Tomaremos que una review recomienda un producto cuando supuntaje sea mayor o igual a 3 estrellas

In [ ]:
#paso valores a bool
dataset = dataset.astype({"recomendable": bool})

In [ ]:
dataset

Entrenamos el modelo con los nuevos datos

In [ ]:
X = dataset['review_title']
y = dataset['recomendable']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
clf_bin = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [ ]:
clf_bin.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf_bin.predict(X_test)

In [ ]:
confusion(y_test,y_pred)

Ya viendo la matriz de confusión ya se ve una mejora pero veamos que dice el score

In [ ]:
clf_bin.score(X_test,y_test)

In [ ]:
dataset.loc[dataset[dataset['stars']< 3].index,'recomendable'] = 0

In [ ]:
dataset_bin = dataset.copy()

In [ ]:
dataset_bin['recom_pred'] = clf_bin.predict(X)

In [ ]:
dataset_bin

Graficamos como quedo la predicción de si las review recomiendan el producto o no

In [ ]:
sns.barplot(dataset_bin['recom_pred'].value_counts().index,dataset_bin['recom_pred'].value_counts().values)
plt.show()

# Reviews negativas relacionadas con envio
<br>
Tambien nos preguntamos ¿Cuantas reviews de productos tienen un puntaje negativo debido a demoras o problemas ne la entrega del producto?

Usamos algoritmos <code>Word2Vec</code> y <code>FastText</code> para medir Word Mover’s Distance o <code>WMD</code>. Esto nos va a servir para obtener la distancia entre palabras u oraciones y distinguir cuales se refieren a entregas rapidas y lentas.

### Modelo FastText con sbwc

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'fasttext-sbwc.3.6.e20.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [ ]:
wordvectors.most_similar_cosmul(positive=['demora'])#probamos palabras cercanas a demora

Probamos como funciona

In [ ]:
#frases distintas
frase1="el paquete llego rapido"
frase2="hubo demora en el envio"

distancia = wordvectors.wmdistance(frase1, frase2)
print('Distancia = %.4f' % distancia)

In [ ]:
#Frases parecidas
frase1="el paquete llego con demora"
frase2="hubo demora en el envio"

distancia = wordvectors.wmdistance(frase1, frase2)
print('Distancia = %.4f' % distancia)

A pesar de que el ultimo par de oraciones tiene una similitud el algoritmo lo marca como lejano

### Modelo w2v con sbwc

Probemos como se desempeña con W2V

In [ ]:
SBW_vectors_file = 'SBW-vectors-300-min5.txt'

In [ ]:
modelow2v = KeyedVectors.load_word2vec_format(SBW_vectors_file, limit=cantidad)

In [ ]:
#frases distintas
frase1= "No han entregado el producto"
frase2= "el envio llego rapido"

distance = modelow2v.wmdistance(frase1, frase2)
print('distance = %.4f' % distance)

In [ ]:
#frases parecidas
frase1= "Plazo de envio no cumplido"
frase2= "hubo demora en el envio"

distance = modelow2v.wmdistance(frase1, frase2)
print('distance = %.4f' % distance)

Como pueden ver, a pesar de que el corpus con el que se entrenó sea en español y el modelo está pre entrenado para funcionar en español los resultados distan de ser buenos para empezar a clasificar una review está relacionada con una entrega.
Sería interesante probar los resultados con un dataset en ingles, donde probablemente sea mejor el desempeño.

Por eso mismo probamos lemmatizando y haciendo busquedas tradicionales

### Identificar reviews negativas relacionadas con envio usando lemmatización
<br>
El objetivo sigue siendo encontrar las reviews que tienen mala puntuación y esté relacionada a la entrega del producto

Traemos las reviews que contengan la palabra <code>"entrega"</code> o <code>"envio"</code>

In [ ]:
datasetprueba = dataset_bin[dataset_bin['review_title'].str.contains("entrega") | dataset_bin['review_title'].str.contains("envio")]
datasetprueba

Podemos ver que el resultado es bastante bajo. ¿Que sucede si normalizamos y lemmatizamos?

In [ ]:
dataset_review_lemma = [] #lista donde se van a guardar las reviews lemmatizadas
for titulo in dataset_bin.review_body:
    dataset_review_lemma.append(" ".join(token_cleaner(titulo)))

In [ ]:
dataset_bin.insert(4,"review_title_lemma", dataset_review_lemma) #agregamos las reviews lemmatizadas al dataframe
dataset_bin

Hacemos las busqueda en base a palabras que estén relacionadas con la entrega y la demor del producto luego de lematizar.
<br>
Hay palabras que están incompletas para tambien se cuenten las conjugación de estas mismas que la lemmatización no pudo reducir.

In [ ]:
palabras_clave = ["entreg","envi","lento","demora","tardar"]#lista de palabras 
dataset_envios_tarde = dataset_bin[(dataset_bin['review_title_lemma'].str.contains('|'.join(palabras_clave))) & (dataset_bin.recomendable==False)]
dataset_envios_tarde

El resultado encontrado fue casi 7 mayor una vez lematizados los <code>review_body</code> de cada review.
<br>
Lematizamos <code>review_body</code> ya que luego de pruebas daba mejores resultados que <code>review_titley</code>

Para poder visualizar un poco mejor vamos a graficar

In [ ]:
valores=[dataset_envios_tarde.shape[0],dataset_bin[dataset_bin["recomendable"]==0].shape[0]-dataset_envios_tarde.shape[0]
-dataset_envios_tarde.shape[0]]

colores = ['#DD7596', '#8EB897']# Colores para el grafico

ax = plt.subplots(figsize=[10,6])
plt.pie(valores,labels=["Reviews negativas relacionadas con envio","Resto de reviews negativas"],
        labeldistance=1.15, wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'black' }, colors=colores,
        explode=(0.2, 0),autopct="%.1f%%");

Encontramos aproximadamente 7119 (9% de las 200.000 reviews) reviews negativas que se relacionan con la entrega del producto.
<br>
Este dato puede servir para analizar y mejorar la distribución de los productos, reduciendo la cantidad de clientes insatisfechos.